In [1]:
#Library imports
import os
import pandas as pd
import numpy as np

import torch
import torch.nn as nn  
import torch.optim as optim  
import torchvision.transforms as transforms  
import torchvision
from torch.utils.data import (Dataset,DataLoader) 

from PIL import Image
from skimage import io

import time

In [2]:
#Get working directory
directory = os.getcwd() # Path of the current working directory.
print(directory)
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f" {device} being used")
#if cpu is being used with PyTorch training will be slower

C:\Users\Andreas\Desktop\AMLS_22-23_SN18086046
 cpu being used


In [3]:
#Hyperparameters
num_epochs = 5 #complete passess over the data set
num_classes = 2 #male/female or smile/no-smile
batch_size = 200 #data must be loaded in batches for more efficient training (high batch size can lead to memory overload)
learning_rate = 0.01

In [4]:
#transformations on the images
#transformer=transforms.Compose([transforms.Resize((224,224)), #make sure all images are the same size
#                                
#                                transforms.RandomHorizontalFlip(), #0.5 chance for image to be flipped to increase uniqueness x2
#                                transforms.ToTensor(),#pixel rate from 0-255 to 0-1 as PyTorch only reads tensors
#                                transforms.Normalize([0.5,0.5,0.5], #change range from 0-1 to (-1)-1 for RGB channel
#                                                     [0.5,0.5,0.5]) #formula:(x-mean)/std
#]) 

In [26]:
# class to load the custom dataset

class celebaDataset(Dataset):
    def __init__(self,csv_file,root_dir,transform=None):
        self.annotations=pd.read_csv(csv_file)
        self.root_dir=root_dir
        self.transform=transform
        pass
  
    def __len__(self):
        
        return len(self.annotations)
        pass
    def __getitem__(self, index): #PyTorch chooses the index
        img_path=os.path.join(self.root_dir, self.annotations.iloc[index,0]) #row 'index' and column 0
        image=io.imread(img_path)
        y_label=torch.tensor(int(self.annotations.iloc[index,1])) #gender/smile label
        y=(y_label+1)/2 #get 0 or 1
        yt=y.type(torch.LongTensor) #change the datatype to tensor as PyTorch get only tensors
        if self.transform:
            image=self.transform(image)
        return (image,yt)
        
        pass

In [28]:

dataset=celebaDataset(csv_file='dataset_AMLS_22-23/celeba/gender_labels.csv',
                      root_dir='dataset_AMLS_22-23/celeba/img',transform=transforms.ToTensor())                                                                                            
                                                                                            
#split the images into test and validation sets
train_dataset,validation_dataset=torch.utils.data.random_split(dataset,[4000,1000]) 
train_loader=DataLoader(dataset=train_dataset,batch_size=200,shuffle=True)
validation_loader=DataLoader(dataset=validation_dataset,batch_size=200,shuffle=False)
#shuffle=True to ensure our model is not biased for some categories 
print('Train dataset size:', len(train_dataset)) #must give 4000
print('Validation dataset size:',len(validation_dataset)) #must give 1000
print(batch_size)


Train dataset size: 4000
Validation dataset size: 1000
200


In [29]:
#CNN model
class convNet(nn.Module):
    def __init__(self,num_classes=2):
        super(convNet,self).__init__()
        
        #Input shape =(batch_size,RGB_channel,Image dimensions)=(50,3,178,218)
        #Layer 1
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=(3,3),stride=(1,1),padding=(1,1))
        #New shape from formula (width-kernel+2P)/s +1 ->Shape=(50,12,178,218)
        self.bn1=nn.BatchNorm2d(num_features=12) #Batch normalisation
        #Shape=(50,12,178,218)
        self.relu1=nn.ReLU()
        #Shape=(50,12,178,218)
        
        self.pool=nn.MaxPool2d(kernel_size=(2,2))
        #Reduce the image size be factor 2
        #Shape= (50,12,89,109)
        
        #Layer 2
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=(3,3),stride=(1,1),padding=(1,1))
        #Shape= (50,20,89,109)
        self.relu2=nn.ReLU()
        #Shape= (50,20,89,109)
        #self.pool2=nn.MaxPool2d(kernel_size=(2,2))
        
        #Layer 3
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=(3,3),stride=(1,1),padding=(1,1))
        #Shape= (50,32,44.5,54.5)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (50,32,44.5,54.5)
        self.relu3=nn.ReLU()
        #Shape= (50,32,44.5,54.5)
        
        
        #fully connected layer
        self.fc=nn.Linear(in_features=32*40*40,out_features=num_classes)
        
        
        #feed forward function
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
        #output=self.pool2(output)
        
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
                
        #Above output will be in matrix form, with shape (50,32,112,112) 
            
        output=output.view(-1,32*40*40)
            
            
        output=self.fc(output)
            
        return output
            

In [30]:
model=convNet(num_classes=2).to(device) #send it to cuda/cpu

In [31]:
#Loss and optimizer functions
optimizer=torch.optim.Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
criterion=nn.CrossEntropyLoss() #Loss function

In [32]:
# Train Network
start_time=time.time()

for epoch in range(num_epochs):
    losses = []

    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward pass (Evaluation)
        output = model(data)
        loss = criterion(output, targets)

        losses.append(loss.item())

        # backward pass (Optimization)
        optimizer.zero_grad()
        loss.backward()

        # adam step
        optimizer.step()

    print(f"Cost at epoch {epoch} is {sum(losses)/len(losses)}")

# Check accuracy on training to see how good our model is
def check_accuracy(loader, model):
    num_correct = 0
    num_total= 0
    
    model.eval()
    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device=device)
            labbels = labels.to(device=device)

            output = model(images)
            _, predictions = output.max(1)
            num_correct += (predictions == labels).sum()
            num_total += predictions.size(0)

        print( f" {num_correct} / {num_total} correct images with accuracy {float(num_correct)/float(num_total)*100:.2f}%")

    model.train()


print("Training set accuracy....")
check_accuracy(train_loader, model)
    
print("Validation set accuracy....")
check_accuracy(validation_loader, model)  

elapsed_time=time.time()
print('Time taken:',(elapsed_time-start_time),'seconds')
    

TypeError: Unexpected type <class 'numpy.ndarray'>